In [10]:
import os
from datetime import datetime, timedelta
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

### Task 1

Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv и experiment_users.csv.

В качестве ответа введите p-value, округлённое до 3-го знака после точки.

In [9]:
URL_BASE = 'https://raw.githubusercontent.com/ElenaNKn/simulator_ab_tests/refs/heads/master/tasks_solution/seminar2/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

In [2]:
os.path.join(URL_BASE, '2022-04-01T12_df_sales.csv')

'https://raw.githubusercontent.com/ElenaNKn/simulator_ab_tests/refs/heads/master/tasks_solution/seminar2/2022-04-01T12_df_sales.csv'

In [3]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_users = read_database('experiment_users.csv')

In [4]:
begin_date = datetime(2022, 3, 16)
end_date = datetime(2022, 3, 23)
df_metrics = (
    df_sales
    [(df_sales['date'] >= begin_date) & (df_sales['date'] < end_date)]
    .groupby('user_id')[['price']].sum()
    .reset_index()  
)

In [5]:
df = pd.merge(df_users, df_metrics, on='user_id', how='left').fillna(0)

In [8]:
data_control = df[df['pilot'] == 0]['price']
data_pilot = df[df['pilot'] == 1]['price']
stats.ttest_ind(data_control, data_pilot)

TtestResult(statistic=np.float64(-1.2837567415000515), pvalue=np.float64(0.19923983306424942), df=np.float64(23331.0))

In [27]:
stats.ttest_ind(data_control, data_pilot).pvalue.item()

0.19923983306424942

### Task 2

При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv.

В качестве ответа введите среднее количество дней между покупками, округлённое до целого значения.

In [11]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [16]:
count_purch = df_sales.groupby('user_id').agg(
    n_purch=('sale_id', 'nunique')
).reset_index()

In [19]:
result = df_sales.merge(count_purch, how='inner', on='user_id')
result = result[result.n_purch >= 2][['user_id', 'date']].sort_values(['user_id', 'date'])

In [20]:
time_differences = result.groupby('user_id')['date'].diff().shift(-1)

In [22]:
time_differences.mean()

Timedelta('17 days 07:37:57.653431857')

### Task 3

Для проверки статистической значимости нужно вычислить p-value и сравнить его с уровнем значимости.

Напишите функцию get_ttest_pvalue. 

Шаблон решения

import numpy as np
from scipy import stats


def get_ttest_pvalue(metrics_a_group, metrics_b_group):
    """Применяет тест Стьюдента, возвращает pvalue.

    :param metrics_a_group (np.array): массив значений метрик группы A
    :param metrics_a_group (np.array): массив значений метрик группы B
    :return (float): значение p-value
    """
    # YOUR_CODE_HERE

In [28]:
import numpy as np
from scipy import stats


def get_ttest_pvalue(metrics_a_group, metrics_b_group):
    """Применяет тест Стьюдента, возвращает pvalue.

    :param metrics_a_group (np.array): массив значений метрик группы A
    :param metrics_a_group (np.array): массив значений метрик группы B
    :return (float): значение p-value
    """
    return stats.ttest_ind(metrics_a_group, metrics_b_group).pvalue.item()